## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [26]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [27]:
# 데이터를 불러옵니다.
base_path = '../../data/'
train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')
print(train.shape, test.shape, submission.shape)

(101763, 23) (67842, 22) (67842, 2)


## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [28]:
train.columns

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')

### 3. 전처리

#### 결측치 처리

### 4. 학습 데이터 분할

In [29]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

# for OOF-prediction split 5% of data as validation dataset.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61, stratify=y)

In [30]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(81410, 22) (81410,) (20353, 22) (20353,)
0.2266429185603734 0.22664963396059548


### 5. 학습 및 평가

In [31]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)
model = XGBClassifier(
    booster='gbtree',
    objective='binary:logistic',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    n_jobs=-1,
    random_state=61
)

In [32]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)


Fitting LightGBM...


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=61, ...)

In [33]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = roc_auc_score

In [34]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)


train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.6810
Validation Score : 0.6668


### 6. Hyper-parameter Tuning

In [35]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 2, 5)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0.5, 0.8)
    reg_lambda = trial.suggest_float('reg_lambda', 1.0, 2.0)
    n_estimators = trial.suggest_int('n_estimators', 20, 50)
    learning_rate = trial.suggest_float('learning_rate', 0.1, 0.2)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = XGBClassifier(
        max_depth=max_depth,
        colsample_bynode=colsample_bynode,
        reg_lambda=reg_lambda,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=61,
        eval_metric=evaluation_metric,
    )

    # K-Fold Cross validation을 구현합니다.
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    losses = []

    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)


    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [36]:
K = 5   # Kfold 수
opt_func = partial(optimizer, X=X, y=y, K=K)

study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=50)

[I 2023-10-15 01:22:27,077] A new study created in memory with name: no-name-3699be1f-afca-4f0d-8065-4689c366ebd9
[I 2023-10-15 01:22:31,361] Trial 0 finished with value: 0.6658877458138204 and parameters: {'max_depth': 3, 'colsample_bynode': 0.6826054729742932, 'reg_lambda': 1.37999505188472, 'n_estimators': 40, 'learning_rate': 0.16608970727170508}. Best is trial 0 with value: 0.6658877458138204.
[I 2023-10-15 01:22:35,613] Trial 1 finished with value: 0.6658062204338558 and parameters: {'max_depth': 3, 'colsample_bynode': 0.7220550286118781, 'reg_lambda': 1.046172467583564, 'n_estimators': 39, 'learning_rate': 0.12025817793581041}. Best is trial 0 with value: 0.6658877458138204.
[I 2023-10-15 01:22:38,308] Trial 2 finished with value: 0.6631236671505308 and parameters: {'max_depth': 2, 'colsample_bynode': 0.7295769414169548, 'reg_lambda': 1.1831127173183544, 'n_estimators': 25, 'learning_rate': 0.1784334916139706}. Best is trial 0 with value: 0.6658877458138204.
[I 2023-10-15 01:22:

In [37]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bynode,params_learning_rate,params_max_depth,params_n_estimators,params_reg_lambda,state
0,0,0.665888,2023-10-15 01:22:27.079930,2023-10-15 01:22:31.361099,0 days 00:00:04.281169,0.682605,0.166090,3,40,1.379995,COMPLETE
1,1,0.665806,2023-10-15 01:22:31.362099,2023-10-15 01:22:35.612898,0 days 00:00:04.250799,0.722055,0.120258,3,39,1.046172,COMPLETE
2,2,0.663124,2023-10-15 01:22:35.614912,2023-10-15 01:22:38.308670,0 days 00:00:02.693758,0.729577,0.178433,2,25,1.183113,COMPLETE
3,3,0.666686,2023-10-15 01:22:38.309670,2023-10-15 01:22:43.897503,0 days 00:00:05.587833,0.672891,0.172849,4,27,1.269556,COMPLETE
4,4,0.665368,2023-10-15 01:22:43.900506,2023-10-15 01:22:48.634027,0 days 00:00:04.733521,0.716007,0.155306,3,33,1.343370,COMPLETE
5,5,0.666669,2023-10-15 01:22:48.636093,2023-10-15 01:22:54.884510,0 days 00:00:06.248417,0.515208,0.178433,4,43,1.795466,COMPLETE
6,6,0.666077,2023-10-15 01:22:54.886920,2023-10-15 01:23:01.487437,0 days 00:00:06.600517,0.713225,0.128149,4,40,1.037933,COMPLETE
7,7,0.665958,2023-10-15 01:23:01.488437,2023-10-15 01:23:08.999446,0 days 00:00:07.511009,0.765532,0.164896,5,32,1.896622,COMPLETE
8,8,0.666132,2023-10-15 01:23:09.001821,2023-10-15 01:23:15.104659,0 days 00:00:06.102838,0.679765,0.111914,5,31,1.456184,COMPLETE
9,9,0.662499,2023-10-15 01:23:15.105658,2023-10-15 01:23:19.172947,0 days 00:00:04.067289,0.771657,0.104610,2,42,1.381921,COMPLETE


In [38]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.6675
Best params:  {'max_depth': 5, 'colsample_bynode': 0.6028784042231218, 'reg_lambda': 1.9107959408881667, 'n_estimators': 38, 'learning_rate': 0.19089614460186538}


In [39]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [40]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [41]:
# Make KFold OOF prediction
def oof_preds(best_model):

    # make KFold
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    final_preds = []
    losses = []
    oof = np.full(len(X), np.nan)
    # fitting with best_model
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        print(f"========== Fold {i+1} ==========")
        best_model.fit(X_train, y_train)
        preds = best_model.predict_proba(X_val)[:, 1]
        oof[val_idx] = preds
        test_preds = best_model.predict_proba(test)[:, 1]
        final_preds.append(test_preds)
        loss = evaluation_metric(y_val, preds)

        losses.append(loss)

    avg_loss = np.mean(losses)
    print(f"Loss : {avg_loss:.4f}")
    return final_preds, oof

In [42]:
test.info() # 결측치 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67842 entries, 0 to 67841
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67842 non-null  int64  
 1   loc                67842 non-null  float64
 2   v(g)               67842 non-null  float64
 3   ev(g)              67842 non-null  float64
 4   iv(g)              67842 non-null  float64
 5   n                  67842 non-null  float64
 6   v                  67842 non-null  float64
 7   l                  67842 non-null  float64
 8   d                  67842 non-null  float64
 9   i                  67842 non-null  float64
 10  e                  67842 non-null  float64
 11  b                  67842 non-null  float64
 12  t                  67842 non-null  float64
 13  lOCode             67842 non-null  int64  
 14  lOComment          67842 non-null  int64  
 15  lOBlank            67842 non-null  int64  
 16  locCodeAndComment  678

In [43]:
## X_test 만들기 : 앞서했던 전처리를 동일하게 적용해주면 됨.


In [44]:
best_params = study.best_trial.params

# define best model
best_model = XGBClassifier(**best_params, random_state=61, eval_metric=evaluation_metric)

# model finalization : 가장 일반적으로 좋은 예측 성능을 냈던 모델로, 전체 데이터 트레이닝.

preds, oof = oof_preds(best_model)
preds = np.mean(preds, axis=0)
preds

========== Fold 1 ==========
========== Fold 2 ==========
========== Fold 3 ==========
========== Fold 4 ==========
========== Fold 5 ==========
Loss : 0.7916


array([0.2593623 , 0.23225331, 0.64655656, ..., 0.18655631, 0.11410312,
       0.7880367 ], dtype=float32)

In [45]:
submission['defects'] = preds

In [46]:
submission.to_csv(base_path+'preds/xgboost.csv', index=False)

In [47]:
oof_df = pd.DataFrame({
    'xgboost_oof': oof
})
oof_df.head()

,xgboost_oof
0,0.071362
1,0.053843
2,0.045884
3,0.085829
4,0.112108


In [48]:
oof_df.to_csv(base_path+'oof/xgboost.csv', index=False)